In [1]:
#Import Packages

import numpy as np
from numpy import random as rd
from pykrx import stock
import scipy as sp
import pandas as pd
from scipy import optimize

In [90]:
#Load Data

ETF_ticker = ["002810", "003920", "004380", "011280", "016580", "007340", "000370", "001510", "033270", "004710", "001500"]
N = len(ETF_ticker)
stocks_data = []
srt = "20010101"
end = "20191231"
for i in range(0, N):
    stocks_data.append(stock.get_market_ohlcv_by_date(srt, end, ETF_ticker[i])["종가"])
    
data = pd.DataFrame(np.asarray(stocks_data).T, columns=ETF_ticker, dtype=np.int32)

In [91]:
N

11

In [4]:
data

,002810,003920,004380,011280,016580,007340,000370,001510,033270,004710,001500
0,1507,252000,727,335,2612,3678,13502,2081,2110,5781,2468
1,1541,254000,731,326,2622,3678,13502,2150,2230,6641,2488
2,1591,252000,757,350,2602,3694,14195,2467,2430,7620,2859
3,1558,253000,746,353,2492,3678,16110,2833,2385,7650,3159
4,1541,255000,755,357,2652,3757,16538,3255,2350,7650,3135
...,...,...,...,...,...,...,...,...,...,...,...
4690,15709,425000,11500,4130,15800,29450,2850,603,18500,8750,10350
4691,15852,436000,11350,4100,15450,29500,2860,600,18300,8240,10350
4692,15900,433000,11500,4290,15500,29600,2840,607,18500,8750,10500
4693,16050,431000,11700,4355,15400,29350,2820,607,18250,8800,10100


In [28]:
#Global variable
var_cov_mat = np.identity(N)
rf = 0.015
ret = pd.DataFrame(np.nan, index=np.arange(data.shape[0]), columns=data.columns)

In [24]:
#Lagrange Multiplier

def eq_constraint(w): # w값들의 합이 1이 되어야 한다.
    summation = 0
    for i in w:
        summation += i
    return summation -1

def flarray(w): #샤프지수가 최대가 되게하는 목적함수
    global var_cov_mat
    
    ret_exd = np.cumprod(ret) - 0.015
    return -(np.dot(np.transpose(ret_exd.iloc[-1]), w) / (np.dot(np.dot(np.transpose(w), var_cov_mat), w))**(1/2))

def get_W_Max_Sharp():
    return sp.optimize.fmin_slsqp(flarray, [1/N for i in range(N)], bounds=[(0,1) for i in range(N)], eqcons=[eq_constraint])

In [31]:
def w_optima(data, day, beforehand):
    data_ = data.iloc[day-beforehand:day]

    global ret

    for i in range(1, len(stocks_data[0])):
        ret.iloc[i] = data.iloc[i]/data.iloc[i-1]
    
    global var_cov_mat
    
    for i in range(0, N):
        for j in range(0, N):
            var_cov_mat[i][j] = np.cov(ret.iloc[1:,i], ret.iloc[1:,j])[0,1]
    return get_W_Max_Sharp()

#w_optima(data,100,30)

In [34]:
print(np.round(w_optima(data,200,30), 5))
print(np.round(w_optima(data,200,120), 5))


Optimization terminated successfully.    (Exit mode 0)
            Current function value: -803.7172378958029
            Iterations: 17
            Function evaluations: 229
            Gradient evaluations: 16
[0.24295 0.      0.25144 0.19293 0.03608 0.23031 0.      0.      0.04629
 0.      0.     ]
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -803.7172378958029
            Iterations: 17
            Function evaluations: 229
            Gradient evaluations: 16
[0.24295 0.      0.25144 0.19293 0.03608 0.23031 0.      0.      0.04629
 0.      0.     ]


In [97]:
#var - cov matrix 생성
def getCov(a, b):
    return np.cov(a,b)


var_cov_mat_ = np.identity(N)

for i in range(0,11):
    for j in range(0,11):
        var_cov_mat_[i][j] = np.cov(ret.iloc[1:,i],ret.iloc[1:,j])[0,1]
np.round(var_cov_mat_*10000,1)

array([[ 4.8,  0.5,  1.1,  1.2,  0.8,  0.5,  1.4,  1.6,  1.2,  1.9,  1.5],
       [ 0.5,  4.7,  0.5,  0.8,  0.5,  0.6,  1.1,  1.2,  0.8,  1.3,  1.1],
       [ 1.1,  0.5,  8.2,  1.4,  0.9,  0.7,  1.4,  1.8,  1.2,  2.3,  1.6],
       [ 1.2,  0.8,  1.4,  6.9,  1.1,  0.7,  1.8,  2.3,  1.5,  2.5,  2.1],
       [ 0.8,  0.5,  0.9,  1.1,  5.4,  0.6,  1. ,  1.3,  1.9,  1.4,  1.1],
       [ 0.5,  0.6,  0.7,  0.7,  0.6,  4.1,  1.2,  1.1,  0.7,  1.2,  1.1],
       [ 1.4,  1.1,  1.4,  1.8,  1. ,  1.2, 10.3,  3.8,  1.7,  3.1,  3.5],
       [ 1.6,  1.2,  1.8,  2.3,  1.3,  1.1,  3.8, 11.4,  2.3,  4.5,  5.5],
       [ 1.2,  0.8,  1.2,  1.5,  1.9,  0.7,  1.7,  2.3,  9.1,  2.3,  1.9],
       [ 1.9,  1.3,  2.3,  2.5,  1.4,  1.2,  3.1,  4.5,  2.3, 13.7,  3.7],
       [ 1.5,  1.1,  1.6,  2.1,  1.1,  1.1,  3.5,  5.5,  1.9,  3.7,  7.8]])

In [78]:
ret.iloc[1:,10]

1       1.008104
2       1.149116
3       1.104932
4       0.992403
5       0.990112
          ...   
4690    0.985714
4691    1.000000
4692    1.014493
4693    0.961905
4694    0.995050
Name: 001500, Length: 4694, dtype: float64

In [69]:
getCov(ret.iloc[1:,1],ret.iloc[1:,2])

array([[4.66633372e-04, 4.56347388e-05],
       [4.56347388e-05, 8.18791749e-04]])

In [73]:
np.cov(ret.iloc[1:,1],ret.iloc[1:,6])

array([[0.00046663, 0.00010932],
       [0.00010932, 0.00102994]])

In [43]:
len(ret[100:150])

50

In [22]:
a = 3
def ff():
    global a
    a = 4
ff()
print(a)

4
